In [3]:
import cv2
import numpy as np
from keras.models import load_model
from numpy import expand_dims
from numpy import reshape
from numpy import load
from numpy import max
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle
import dlib
from PIL import Image

In [4]:
face_model = load_model('faceRecognition Model/facenet_keras.h5')
with open('faceRecognition Model/svm_model.pkl', 'rb') as file:
    model_svm = pickle.load(file)

In [5]:
def detect_faces_hog(image):
    pixels = np.asarray(image)
    # print("Pixels shape:", pixels.shape)
    detector = dlib.get_frontal_face_detector()
    dets = detector(pixels, 1)
    # print("Detections:", dets)

    if len(dets) == 0:
        return None

    x1_face, y1_face, x2_face, y2_face = dets[0].left(), dets[0].top(), dets[0].right(), dets[0].bottom()

    print("Face coordinates (x1, y1, x2, y2):", x1_face, y1_face, x2_face, y2_face)

    # Check if the detected face coordinates are within the image boundaries
    if x1_face > 0:
        x1_face = x1_face
    else: 
        x1_face = 0

    # y1_face = max(0, y1_face)
    if y1_face > 0:
        y1_face = y1_face
    else:
        y1_face = 0

    # x2_face = min(pixels.shape[1], x2_face)
    if x2_face < pixels.shape[1]:
        x2_face = x2_face
    else:
        x2_face = pixels.shape[1]
    
    # y2_face = min(pixels.shape[0], y2_face)
    if y2_face < pixels.shape[0]:
        y2_face = y2_face
    else:
        y2_face = pixels.shape[0]


    print("Cropped face coordinates (x1, y1, x2, y2):", x1_face, y1_face, x2_face, y2_face)

    if x1_face >= x2_face or y1_face >= y2_face:
        return None

    store_face = pixels[y1_face:y2_face, x1_face:x2_face]
    image1 = Image.fromarray(store_face, 'RGB')   
    image1 = image1.resize((160, 160))            
    face_array = np.asarray(image1)              

    return face_array

In [6]:
def extract_embeddings(model,face_pixels):
  face_pixels = face_pixels.astype('float32') 
  mean = face_pixels.mean()                   
  std  = face_pixels.std()                     
  face_pixels = (face_pixels - mean)/std
  samples = np.expand_dims(face_pixels,axis=0)   
  yhat = model.predict(samples)
  return yhat[0]

In [7]:
out_encode = LabelEncoder()
persons = ['Abdelghafour', 'Ania', 'chris_evans', 'chris_hemsworth', 'Karima', 'mark_ruffalo', 'miko', 'robert_downey_jr', 'scarlett_johansson']
out_encode.fit(persons)

LabelEncoder()

In [8]:
persons_dict = {i: name for i, name in enumerate(persons)}


In [11]:
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()

    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    face = detect_faces_hog(rgb_frame)

    if face is not None:
        face = face.reshape(160, 160, 3)

        embeddings = extract_embeddings(face_model, face)

        in_encode = Normalizer(norm='l2')
        flattened_embeddings = embeddings.reshape(1, -1)
        normalized_embeddings = in_encode.transform(flattened_embeddings)

        predict_test = model_svm.predict(normalized_embeddings)
        probability = model_svm.predict_proba(normalized_embeddings)
        confidence = np.max(probability)

        predicted_class_index = out_encode.inverse_transform(predict_test)[0]
        print("Predicted class index:", predicted_class_index)
        predicted_class_key = f'class_{predicted_class_index}'


        if confidence > 0.3:
            cv2.putText(frame, f'Student: {predicted_class_index}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
             cv2.putText(frame, 'Prediction: Unknown', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):   
        break

camera.release() 
cv2.destroyAllWindows()

Face coordinates (x1, y1, x2, y2): 259 175 366 283
Cropped face coordinates (x1, y1, x2, y2): 259 175 366 283
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 259 175 366 283
Cropped face coordinates (x1, y1, x2, y2): 259 175 366 283
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 259 175 366 283
Cropped face coordinates (x1, y1, x2, y2): 259 175 366 283
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 253 168 382 297
Cropped face coordinates (x1, y1, x2, y2): 253 168 382 297
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 259 175 366 283
Cropped face coordinates (x1, y1, x2, y2): 259 175 366 283
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 259 175 366 283
Cropped face coordinates (x1, y1, x2, y2): 259 175 366 283
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2): 270 163 378 271
Cropped face coordinates (x1, y1, x2, y2): 270 163 378 271
Predicted class index: Ania
Face coordinates (x1, y1, x2, y2):

In [10]:
camera.release() 
cv2.destroyAllWindows()